In [1]:
from models.DistMult import DistMult
import pickle
import torch

In [2]:
with open(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\processed\triplets.pickle', 'rb') as handle:
    data = pickle.load(handle)

with open(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\processed\triplet_lookup.pickle', 'rb') as handle:
    lookup = pickle.load(handle)

with open(r'C:\Users\odhra\Documents\graph_project\allosteric_reasonings\data\processed\ASD_dictionary.pickle', 'rb') as handle:
    ASD_dictionary = pickle.load(handle)

In [3]:
entities = len(lookup) 

In [4]:
model = DistMult(num_entities = entities, embedding_dim=2)
print(model)

DistMult(
  (emb_e): Embedding(70551, 2, padding_idx=0)
  (emb_rel): Embedding(3, 2, padding_idx=0)
  (inp_drop): Dropout(p=0.9, inplace=False)
  (loss): CrossEntropyLoss()
)


In [5]:
params = list(model.parameters())
len(params)
params[1].size()

torch.Size([3, 2])

In [40]:
objects = torch.LongTensor([6000,200,70000,5000])
relationships = torch.LongTensor([1,2,1,1])
print(objects)
print(relationships)

tensor([ 6000,   200, 70000,  5000])
tensor([1, 2, 1, 1])


In [43]:
pred = model(e1 = objects, rel = relationships)


tensor([[0., -0.],
        [0., 0.],
        [0., 0.],
        [0., -0.]], grad_fn=<MulBackward0>)
tensor([[ 0.0000,  1.1757, -0.3567,  ..., -0.7187,  1.2659, -0.3215],
        [ 0.0000, -0.0183, -1.0365,  ...,  3.2094, -0.5178,  0.1654]],
       grad_fn=<TransposeBackward0>)


In [44]:
print(pred)

tensor([[0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000]],
       grad_fn=<SigmoidBackward>)


In [46]:
#okay so just feed in your embeddings here I guess! 
model.loss()

TypeError: forward() missing 2 required positional arguments: 'input' and 'target'

In [28]:
out.size()
#okay this makes sense its spitting out a prediction!!! 

torch.Size([4, 70551])

tensor([[0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.5000]],
       grad_fn=<SigmoidBackward>)


In [7]:
opt = torch.optim.Adam(model.parameters(), lr=0.003, weight_decay=0.995)

In [8]:
model.train()

DistMult(
  (emb_e): Embedding(70551, 2, padding_idx=0)
  (emb_rel): Embedding(3, 2, padding_idx=0)
  (inp_drop): Dropout(p=0.9, inplace=False)
  (loss): CrossEntropyLoss()
)